# Baseline Experiment
Extracting C++ optimizer and running a short optimization loop.

In [5]:
import json
import os
import subprocess
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree

print(f"Current working directory: {os.getcwd()}")

# Setup paths
KERNEL_PATH = '/home/code/research/kernels/smartmanoj_santa-claude/santa-claude.ipynb'
DATA_DIR = '/home/data'
SUBMISSION_FILE = 'submission.csv'

# 1. Extract C++ code
print("Extracting C++ code...")
if not os.path.exists(KERNEL_PATH):
    print(f"ERROR: Kernel file not found at {KERNEL_PATH}")
else:
    with open(KERNEL_PATH, 'r') as f:
        nb = json.load(f)

    for cell in nb['cells']:
        if cell['cell_type'] == 'code':
            source = ''.join(cell['source'])
            if '%%writefile a.cpp' in source:
                content = source.replace('%%writefile a.cpp\n', '')
                with open('tree_packer.cpp', 'w') as f_out:
                    f_out.write(content)
                print("Extracted tree_packer.cpp")
            if '%%writefile bp.cpp' in source:
                content = source.replace('%%writefile bp.cpp\n', '')
                with open('bp.cpp', 'w') as f_out:
                    f_out.write(content)
                print("Extracted bp.cpp")

    # 2. Compile C++ code
    print("Compiling C++ code...")
    try:
        subprocess.run(['g++', '-O3', '-march=native', '-std=c++17', '-fopenmp', '-o', 'tree_packer', 'tree_packer.cpp'], check=True)
        subprocess.run(['g++', '-O3', '-std=c++17', '-o', 'bp', 'bp.cpp'], check=True)
        print("Compilation successful.")
    except subprocess.CalledProcessError as e:
        print(f"Compilation failed: {e}")

# 3. Define Model Class
class CppOptimizer:
    def __init__(self, iterations=5000, restarts=16):
        self.iterations = iterations
        self.restarts = restarts
        
    def optimize(self, input_file, output_file):
        print(f"Running optimization with {self.iterations} iterations and {self.restarts} restarts...")
        # Run tree_packer
        subprocess.run(['./tree_packer', '-i', input_file, '-o', 'submission_opt.csv', '-n', str(self.iterations), '-r', str(self.restarts)], check=True)
        # Run BP
        subprocess.run(['./bp', 'submission_opt.csv', output_file], check=True)
        # Run tree_packer again
        subprocess.run(['./tree_packer', '-i', output_file, '-o', 'submission_opt.csv', '-n', str(self.iterations), '-r', str(self.restarts)], check=True)
        # Final copy
        subprocess.run(['cp', 'submission_opt.csv', output_file], check=True)
        print("Optimization finished.")

# 4. Initialize submission
print("Initializing submission...")
subprocess.run(['cp', os.path.join(DATA_DIR, 'sample_submission.csv'), SUBMISSION_FILE], check=True)

# 5. Execute
model = CppOptimizer(iterations=5000, restarts=16)
model.optimize(SUBMISSION_FILE, SUBMISSION_FILE)

Current working directory: /home/code
Extracting C++ code...
Extracted tree_packer.cpp
Extracted bp.cpp
Compiling C++ code...


Compilation successful.
Initializing submission...
Running optimization with 5000 iterations and 16 restarts...


Tree Packer v21 - ENHANCED (26 threads)
NEW: Swap moves, multi-angle restarts, higher SA temperature
Iterations: 5000, Restarts: 16
Processing all n=1..200 concurrently
Loading submission.csv...
Loaded 200 configs
Initial: 173.652299

Phase 1: Parallel optimization...

n=  1: 1.000000 -> 0.661250 (33.8750%)


n=  2: 0.733589 -> 0.451073 (38.5115%)


n=  3: 0.930301 -> 0.435143 (53.2256%)


...

n= 45: 0.615654 -> 0.587636 (from n=46 removal, 4.5510%)


n= 44: 0.608987 -> 0.589515 (from n=45 removal, 3.1974%)


n= 40: 0.631786 -> 0.630257 (from n=41 removal, 0.2422%)


n= 27: 0.634704 -> 0.577475 (from n=28 removal, 9.0166%)
n= 26: 0.609814 -> 0.590344 (from n=27 removal, 3.1928%)


n= 21: 0.578721 -> 0.563540 (from n=22 removal, 2.6232%)
Pass 1 complete, continuing...

Initial: 125.341654
Final:   121.433545
Improve: 3.908109 (3.1180%)
Phase 1 improved: 188 configs
Phase 2 back-prop improved: 77 configs
Time:    837.9s (with 26 threads)
Saved submission_opt.csv
Optimization finished.


In [6]:
# Validation Code
getcontext().prec = 30
scale_factor = 1

class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        """Initializes the Christmas tree with a specific position and rotation."""
        # Strip 's' if present
        self.center_x = Decimal(str(center_x).strip('s'))
        self.center_y = Decimal(str(center_y).strip('s'))
        self.angle = Decimal(str(angle).strip('s'))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                # Start at Tip
                (Decimal('0.0') * scale_factor, tip_y * scale_factor),
                # Right side - Top Tier
                (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
                # Right side - Middle Tier
                (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
                # Right side - Bottom Tier
                (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
                # Right Trunk
                (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
                # Left Trunk
                (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                # Left side - Bottom Tier
                (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                # Left side - Middle Tier
                (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
                # Left side - Top Tier
                (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

def load_configuration_from_df(n: int, df: pd.DataFrame) -> list[ChristmasTree]:
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row["x"])
        y = str(row["y"])
        deg = str(row["deg"])
        if x and y and deg:
            trees.append(ChristmasTree(x, y, deg))
    return trees

def get_score(trees: list[ChristmasTree], n: int) -> float:
    if not trees: return 0.0
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / float(scale_factor) for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    side_length = max(max_x - min_x, max_y - min_y)
    return side_length**2 / n

def has_overlap(trees: list[ChristmasTree]) -> bool:
    if len(trees) <= 1: return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i: continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def score_and_validate_submission(file_path: str, max_n: int = 200) -> dict:
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        return {"status": "FAILED", "error": "File Not Found"}
    
    total_score = 0.0
    failed_overlap_n = []
    
    print(f"--- Scoring and Validation: {file_path} (N=1 to {max_n}) ---")
    for n in range(1, max_n + 1):
        trees = load_configuration_from_df(n, df)
        if trees:
            current_score = get_score(trees, n)
            total_score += current_score
            if has_overlap(trees):
                failed_overlap_n.append(n)
                print(f"  ❌ N={n:03d}: OVERLAP DETECTED! (Score contribution: {current_score:.12f})")
    
    if failed_overlap_n:
        status = "FAILED (Overlaps)"
    else:
        status = "SUCCESS"
        
    print(f"**Total Submission Score (Σ S²/N):   {total_score:.14f}**")
    return {"status": status, "total_score": total_score, "failed_overlap_n": failed_overlap_n}

# Validate
result = score_and_validate_submission(SUBMISSION_FILE)
print(f"Final Result: {result}")


--- Scoring and Validation: submission.csv (N=1 to 200) ---


**Total Submission Score (Σ S²/N):   121.43354547493108**
Final Result: {'status': 'SUCCESS', 'total_score': 121.43354547493108, 'failed_overlap_n': []}
